# Working with Unstructured Grid Data

UXarray offers support for loading and representing unstructured grids
by providing Xarray-like functionality paired with new routines that
are specifically written for operating on unstructured grids.


## Grid Definition and Data Variables

When working with Unstructured Grids, the grid definition and data variables
are often stored as separate files. This means that there are multiple
separate files that need to be read and linked together to represent the
entire dataset.

For example, the following sample dataset is taken from the NOAA Geoflow project,
which is made up of 4 files: 1 grid definition and 3 data files. (Special thanks to John Clyne, Shilpi Gupta, and the VAPOR team for providing this data!)

```
geoflow-small
│   grid.nc
│   v1.nc
│   v2.nc
│   v3.nc
```


## Grid Conventions

Given the complexity of Unstructured Grids, there are many different ways of representing their underlying topology and structure. These representations are referred to as conventions, and they outline
the required connectivity variables, naming conventions, data types, and many other specifications. UXarray uses the [UGRID](http://ugrid-conventions.github.io/ugrid-conventions/)
conventions as a foundation for internally representing Unstructured Grids, converting any supported input grid format into the UGRID convention at the data loading step. Below is a list of supported formats and conventions that can be read in with UXarray:
* UGRID
* Model for Prediction Across Scales (MPAS)
* Exodus

In addition to loading datasets, we also provide support for constructing a grid from user-defined primitives such as vertices, which is showcased in our other notebooks.


## Reading Grid and Data Files
UXarray provides the `UxDataset` data structure, which is an unstructure grid-informed implementation of Xarray's `Dataset` class. The main addition is the introduction of the `uxgrid` property, which stores our grid topology dimensions, coordinates, variables and provides grid-specific functions.

Constructing a `UxDataset` can be done using our custom `open_dataset` and `open_mfdataset` methods, depending on whether one or multiple data files are meant to be linked to a single grid.


In [1]:
import uxarray as ux

In [2]:
# Base data path
base_path = "../../test/meshfiles/ugrid/geoflow-small/"

# Path to Grid file
grid_path = base_path + "grid.nc"

# Paths to Data Variable files
var_names = ['v1.nc', 'v2.nc', 'v3.nc']

data_paths = [base_path + name for name in var_names]

Loading a single data file with a grid is done using the `open_dataset` method. The resulting `UxDataset` only contains the data variables stored in `v1.nc`.

In [3]:
uxds_single = ux.open_dataset(grid_path, data_paths[0])
uxds_single

<xarray.UxDataset>
Dimensions:  (time: 1, meshLayers: 20, nMeshNodes: 6000)
Coordinates:
  * time     (time) float64 13.0
Dimensions without coordinates: meshLayers, nMeshNodes
Data variables:
    v1       (time, meshLayers, nMeshNodes) float64 -0.009766 ... 0.03283

Similarly, if you wish to open multiple data files with a grid, you would use the `open_mfdataset` method. The resulting `UxDataset` contains all the data variables stored in `v1.nc`, `v2.nc`, and `v3.nc`

In [4]:
uxds_complete = ux.open_mfdataset(grid_path, data_paths)
uxds_complete

<xarray.UxDataset>
Dimensions:  (time: 1, meshLayers: 20, nMeshNodes: 6000)
Coordinates:
  * time     (time) float64 13.0
Dimensions without coordinates: meshLayers, nMeshNodes
Data variables:
    v1       (time, meshLayers, nMeshNodes) float64 dask.array<chunksize=(1, 20, 6000), meta=np.ndarray>
    v2       (time, meshLayers, nMeshNodes) float64 dask.array<chunksize=(1, 20, 6000), meta=np.ndarray>
    v3       (time, meshLayers, nMeshNodes) float64 dask.array<chunksize=(1, 20, 6000), meta=np.ndarray>

Each dataset also contains a `uxgrid` propoerty, which represents the grid that the data variables lie on and can be used to execute grid specific functions and access grid topology dimensions, coordinates, and variables. A detailed overview of functionalities can be found in subsequent notebooks.

For both the single and complete instances of `UxDataset`, the `uxgrid` property contains the same grid information, however they are each instantiated separately.


In [5]:
# check if the grids contain the same variables & information
print(uxds_single.uxgrid == uxds_complete.uxgrid)

# check if the grids point to the same object in memory
print(uxds_single.uxgrid is uxds_complete.uxgrid)

True
False


Printing out the `uxgrid` accessor provides an overview of the original grid format, dimensions, coordinates, and connectivity variables.

In [6]:
uxds_complete.uxgrid

<uxarray.Grid>
Original Grid Type: ugrid
Grid Dimensions:
  * nMesh2_face: 3840
  * nMaxMesh2_face_nodes: 4
  * nMesh2_node: 6000
Grid Coordinate Variables:
  * Mesh2_node_x: (6000,)
  * Mesh2_node_y: (6000,)
Grid Connectivity Variables:
  * Mesh2_face_nodes: (3840, 4)
  * nNodes_per_face: (3840,)

These dimensions, coordinates, and connectivity variables can be accessed with attributes using the same names as show in the print out.

In [7]:
uxds_complete.uxgrid.nMesh2_node

6000

In [8]:
uxds_complete.uxgrid.Mesh2_node_x

<xarray.DataArray 'mesh_node_x' (nMeshNodes: 6000)>
[6000 values with dtype=float64]
Coordinates:
    mesh_node_x  (nMeshNodes) float64 ...
    mesh_node_y  (nMeshNodes) float64 ...
Dimensions without coordinates: nMeshNodes
Attributes:
    standard_name:  longitude
    long_name:      Longitude of 2D mesh nodes.
    units:          degrees_east

In [9]:
uxds_complete.uxgrid.Mesh2_face_nodes

<xarray.DataArray 'mesh_face_nodes' (nMeshFaces: 3840, nFaceNodes: 4)>
array([[   0,    1,    6,    5],
       [   1,    2,    7,    6],
       [   2,    3,    8,    7],
       ...,
       [5991, 5992, 5997, 5996],
       [5992, 5993, 5998, 5997],
       [5993, 5994, 5999, 5998]])
Dimensions without coordinates: nMeshFaces, nFaceNodes
Attributes:
    cf_role:      face_node_connectivity
    long_name:    Maps every face to its corner nodes.
    start_index:  0
    _FillValue:   -9223372036854775808